In [1]:
#TO-BE-fixed: Haroun Bensaadi

## Issue: having a problem installing dFBA

In [2]:

from cobra.io import read_sbml_model
from dfba import DfbaModel, ExchangeFlux, KineticVariable
from dfba.plot.plotly import *
import plotly.io as pio


ModuleNotFoundError: No module named 'dfba'

In [2]:
pip install dfba

  Using cached dfba-0.1.8.tar.gz (63 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached symengine-0.11.0-cp39-cp39-win_amd64.whl.metadata (1.2 kB)
Using cached symengine-0.11.0-cp39-cp39-win_amd64.whl (16.7 MB)
Failed to build dfba
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for dfba (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\control.py -> build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\exchange.py -> build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\helpers.py -> build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\jit.py -> build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\library.py -> build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\model.py -> build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\types.py -> build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\variable.py -> build\lib.win-amd64-cpython-39\dfba
      copying src\dfba\_version.py -> build\lib.win-amd64-cpython-39\dfba
     

Dynamic flux based analysis (dFBA) is used to simulate the production of lysine in a batch culture

In [3]:
model = read_sbml_model('Files/iCGB21FR_updated.xml')
model.solver = "glpk"
dfba_model = DfbaModel(model)

biomass = KineticVariable("Biomass")
glucose = KineticVariable("Glucose")
oxygen = KineticVariable("Oxygen")
lysine = KineticVariable("Lysine")

dfba_model.add_kinetic_variables([biomass, glucose, oxygen, lysine])

mu = ExchangeFlux("drain_biomass")
v_G = ExchangeFlux("boundary_glucose")
v_O = ExchangeFlux("boundary_o2_e")
v_L = ExchangeFlux("lysine_e")

dfba_model.add_exchange_fluxes([mu, v_G, v_O, v_L])


dfba_model.add_rhs_expression("Biomass", mu * biomass)
dfba_model.add_rhs_expression("Glucose", v_G * 180.1559 * biomass / 1000.0)
dfba_model.add_rhs_expression("Oxygen", v_O * 16.0 * biomass / 1000.0)
dfba_model.add_rhs_expression("Lysine", v_L * 146.19 * biomass / 1000.0)


dfba_model.add_exchange_flux_lb("boundary_glucose", 10.5*(glucose/(0.0027+glucose)), glucose)
dfba_model.add_exchange_flux_lb("boundary_o2_e", 15.0*(oxygen/(0.024+oxygen)), oxygen)
model.reactions.drain_biomass.lower_bound = 0.1   #[1/h]


dfba_model.add_initial_conditions(
    {
        "Biomass": 0.03, #(gDW/L)
        "Glucose": 15.5, #(g/L)
        "Oxygen": 1,     #(g/L)
        "Lysine" : 0     #(g/L)
    }
)
# # simulate model across interval t = [0.0,25.0](hours) with outputs for plotting every 0.1h and optional list of fluxes

concentrations, trajectories = dfba_model.simulate(0.0, 25.0, 0.01, ["drain_biomass", "boundary_glucose", "boundary_o2_e", "lysine_e"])

pio.templates.default = "plotly_white"

NameError: name 'DfbaModel' is not defined